<a href="https://colab.research.google.com/github/hebaashraf21/KNN_K-means_PCA_CUDA_Implementation/blob/main/knn/knn_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup cuda environment
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-3dj4k1nu
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-3dj4k1nu
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 326b0a57a80c6d0b4bad25ca7adf8138419ef1cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-s9x2i78e/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpb40foodz".


In [51]:
%%writefile knn_gpu.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>
#include <iostream>
#include <chrono>

#define MAX_ERR 1e-6

__global__ void knn_kernel(float *reference_points, float *query_points, int *results, int n, int m, int k, int dim) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < m) {
        float *min_distances = new float[k];
        int *min_indices = new int[k];

        for (int i = 0; i < k; ++i) {
            min_distances[i] = INFINITY;
            min_indices[i] = -1;
        }

        for (int i = 0; i < n; ++i) {
            float distance = 0;
            for (int j = 0; j < dim; ++j) {
                float diff = reference_points[i * dim + j] - query_points[idx * dim + j];
                distance += diff * diff;
            }

            for (int l = 0; l < k; ++l) {
                if (distance < min_distances[l]) {
                    for (int t = k - 1; t > l; --t) {
                        min_distances[t] = min_distances[t - 1];
                        min_indices[t] = min_indices[t - 1];
                    }
                    min_distances[l] = distance;
                    min_indices[l] = i;
                    break;
                }
            }
        }

        for (int i = 0; i < k; ++i) {
            results[idx * k + i] = min_indices[i];
        }

        delete[] min_distances;
        delete[] min_indices;
    }
}

void knn_with_cuda(float *reference_points, float *query_points, int *results, int n, int m, int k, int dim) {
    float *d_reference, *d_query;
    int *d_results;

    cudaMalloc((void **)&d_reference, sizeof(float) * dim * n);
    cudaMalloc((void **)&d_query, sizeof(float) * dim * m);
    cudaMalloc((void **)&d_results, sizeof(int) * m * k);

    cudaMemcpy(d_reference, reference_points, sizeof(float) * dim * n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_query, query_points, sizeof(float) * dim * m, cudaMemcpyHostToDevice);

    int blockSize = 256;
    int numBlocks = (m + blockSize - 1) / blockSize;

    knn_kernel<<<numBlocks, blockSize>>>(d_reference, d_query, d_results, n, m, k, dim);

    cudaMemcpy(results, d_results, sizeof(int) * m * k, cudaMemcpyDeviceToHost);

    cudaFree(d_reference);
    cudaFree(d_query);
    cudaFree(d_results);
}

int main() {
    int n = 1000000; // Number of reference points
    int m = 1000;  // Number of query points
    int k = 5;      // Number of nearest neighbors
    int dim = 3;    // Number of dimensions

    float *reference_points = (float *)malloc(sizeof(float) * dim * n);
    float *query_points = (float *)malloc(sizeof(float) * dim * m);
    int *results = (int *)malloc(sizeof(int) * m * k);

    // Initialize reference_points and query_points with random values
    for (int i = 0; i < dim * n; ++i) {
        reference_points[i] = static_cast<float>(rand()) / RAND_MAX;
    }
    for (int i = 0; i < dim * m; ++i) {
        query_points[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    auto start = std::chrono::high_resolution_clock::now();
    knn_with_cuda(reference_points, query_points, results, n, m, k, dim);
    auto end = std::chrono::high_resolution_clock::now();

    std::chrono::duration<float, std::milli> duration = end - start;
    std::cout << "Time taken with CUDA: " << duration.count() << " milliseconds" << std::endl;

    free(reference_points);
    free(query_points);
    free(results);

    return 0;
}


Overwriting knn_gpu.cu


In [52]:
!nvcc knn_gpu.cu -o knn_gpu
!./knn_gpu

Time taken with CUDA: 0.22591 milliseconds
